In [ ]:
# Importing Libraries
import cv2
import mediapipe as mp
import pandas as pd
import os
import warnings
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
warnings.filterwarnings("ignore")
from IPython.display import display, Javascript
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
# Feature Extraction
tab_features = []
tab_label = []
#directory = '../ML JComp/train/train'
directory = '/content/drive/MyDrive/ML JComp/train'
for folder in os.listdir(directory):
    f = os.path.join(directory, folder)
    for file in os.listdir(f):
        sample_img = cv2.imread(os.path.join(f, file))
        with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:
            #print(sample_img.shape)
            results = hands.process(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))
            image = cv2.imread(os.path.join(f, file))
            image_height, image_width, _ = image.shape
            if results.multi_hand_landmarks:
                x, y, z, label = [], [], [], []
                for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    for i in range(21):
                        x.append(hand_landmarks.landmark[mp_hands.HandLandmark(
                            i).value].x * image_width - hand_landmarks.landmark[mp_hands.HandLandmark(0).value].x * image_width)
                        y.append(hand_landmarks.landmark[mp_hands.HandLandmark(
                            i).value].y * image_height - hand_landmarks.landmark[mp_hands.HandLandmark(0).value].y * image_height)
                        z.append(hand_landmarks.landmark[mp_hands.HandLandmark(
                            i).value].z * image_width - hand_landmarks.landmark[mp_hands.HandLandmark(0).value].z * image_width)
                    col = x + y + z
                tab_features.append(col)
                tab_label.append(folder)

In [ ]:
df_features = pd.DataFrame(tab_features).iloc[:, :63]
df_label = pd.DataFrame(tab_label)

In [ ]:
# Printing the Dataset
print(df_features)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df_label = label_encoder.fit_transform(df_label)

In [ ]:
# Train Test Split
target = df_label
features = df_features
X_train, X_test, Y_train, Y_test = train_test_split(
    features, target, test_size=0.2, random_state=10)

In [ ]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
rf_pred = rf.predict(X_test)
print(classification_report(rf_pred, Y_test))

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# Function Call
original_image = take_photo()
img = cv2.imread(original_image)
cv2_imshow(img)

In [ ]:
test_tab_features = []
with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:
    results = hands.process(cv2.cvtColor(
        img, cv2.COLOR_BGR2RGB))
    #image = cv2.imread(os.path.join(f, file))
    image_height, image_width, _ = img.shape
    if results.multi_hand_landmarks:
        x, y, z, label = [], [], [], []
        for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
            for i in range(21):
                x.append(hand_landmarks.landmark[mp_hands.HandLandmark(
                    i).value].x * image_width - hand_landmarks.landmark[mp_hands.HandLandmark(0).value].x * image_width)
                y.append(hand_landmarks.landmark[mp_hands.HandLandmark(
                    i).value].y * image_height - hand_landmarks.landmark[mp_hands.HandLandmark(0).value].y * image_height)
                z.append(hand_landmarks.landmark[mp_hands.HandLandmark(
                    i).value].z * image_width - hand_landmarks.landmark[mp_hands.HandLandmark(0).value].z * image_width)
            col = x + y + z
        test_tab_features.append(col)

In [ ]:
rf_pred = rf.predict(test_tab_features)

In [ ]:
tested_gesture = label_encoder.inverse_transform(rf_pred)

In [ ]:
print(tested_gesture[0])